## Import modules

In [1]:
import pandas as pd
import psycopg2
from configparser import ConfigParser
import glob

## Create table in local Postgres database

In [8]:
# Read database configuration from config file
config = ConfigParser()
config.read('config.ini')

# Connect to PostgreSQL database
conn = psycopg2.connect(
    host=config['postgresql']['host'],
    port=config['postgresql']['port'],
    database=config['postgresql']['database'],
    user=config['postgresql']['user'],
    password=config['postgresql']['password']
)

# Read SQL query to create table from file
with open('create_tables.sql', 'r') as f:
    create_table_query = f.read()

# Execute SQL query to create trips table
with conn.cursor() as cursor:
    cursor.execute(create_table_query)

## Import CSV files and clean data

In [5]:
# Define function to clean and transform data
def clean_data(df):
    df['started_at'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d %H:%M:%S')
    df['ended_at'] = pd.to_datetime(df['ended_at'], format='%Y-%m-%d %H:%M:%S')
    df['ride_time'] = pd.to_timedelta(df['ended_at'] - df['started_at'], unit='minutes')
    df = df[df['ride_time'] >= pd.Timedelta(minutes=1)]
    df = df[['ride_id', 'rideable_type', 'started_at', 'ended_at', 'start_station_name',
             'start_station_id', 'start_lat', 'start_lng',
             'end_station_name', 'end_station_id', 'end_lat',
             'end_lng', 'member_casual', 'ride_time']]
    return df

# Read CSV files into pandas dataframes and concatenate into a single dataframe
src_dir = './src/*.csv'
src_files = glob.glob(src_dir)
df_list = []
for file in src_files:
    df = pd.read_csv(file, parse_dates=['started_at','ended_at'])
    df_list.append(df)
df = pd.concat(df_list, ignore_index=True)

# Clean and transform data
df = clean_data(df)

## Write dataframes to database

In [9]:
# Write dataframe to PostgreSQL database
with conn.cursor() as cursor:
    for index, row in df.iterrows():
        cursor.execute('INSERT INTO trips VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)',
                   (row['ride_id'], row['rideable_type'], row['started_at'], row['ended_at'],
                    row['start_station_name'], row['start_station_id'], row['start_lat'],
                    row['start_lng'], row['end_station_name'], row['end_station_id'],
                    row['end_lat'], row['end_lng'], row['member_casual'],
                    row['ride_time']))
    conn.commit()

# Close database connection
conn.close()